In [ ]:
import sys, os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sys.path.append("../python-scripts/")
import functions_common

In [ ]:
parameters = functions_common.read_parameters()
parameters

In [ ]:
macrophages_annotated = pd.read_csv("/home/wgiese/Data/zebrafish_xenograft/04_Processed_Data/01_Annotated_Macrophages/200811_LBT123_1dpi_Pos005.csv", sep =";")
macrophages_annotated

In [ ]:
macrophages_algorithm = pd.read_csv("/home/wgiese/Data/zebrafish_xenograft/output_data/threshold_otsu/macrophage_properties_200811_LBT123_1dpi_Pos005.csv")
macrophages_algorithm

In [ ]:
time_frames = min(len(macrophages_algorithm["time_frame"].unique()),len(macrophages_annotated["time_point"].unique()))
                                  
detection_rates_df = pd.DataFrame()    
    
counter =1

for tp in range(time_frames):
    macrophages_algorithm_ = macrophages_algorithm[macrophages_algorithm["time_frame"] == tp]
    macrophages_annotated_ =macrophages_annotated[macrophages_annotated["time_point"] == tp]
    pairs_df, pairs_df_ = functions_common.get_macrophage_pairs(parameters, macrophages_algorithm_, macrophages_annotated_)
    
    detection_rates_df.at[counter, "time_frame"] = tp
    detection_rates_df.at[counter, "macrophages_algorithm"] = len(macrophages_algorithm_.index)
    detection_rates_df.at[counter, "macrophages_annotated"] = len(macrophages_annotated_.index)
    detection_rates_df.at[counter, "macrophage_pairs"] = len(pairs_df.index)
    
    
    if len(macrophages_algorithm_.index) > 0:
        false_positive_rate = len(pairs_df.index)/len(macrophages_algorithm_.index)
        print(false_positive_rate)
    if len(macrophages_annotated_.index) > 0:
        false_negative_rate = len(pairs_df.index)/len(macrophages_annotated_.index)
        print(false_negative_rate)
        
    detection_rates_df.at[counter, "false_positive_rate"] = 1.0 - false_positive_rate
    detection_rates_df.at[counter, "false_negative_rate"] = 1.0 - false_negative_rate
        
    counter += 1

In [ ]:
detection_rates_df

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot(data = detection_rates_df, x="time_frame", y="macrophages_algorithm", ax= ax)
sns.lineplot(data = detection_rates_df, x="time_frame", y="macrophages_annotated", ax= ax)
sns.lineplot(data = detection_rates_df, x="time_frame", y="macrophage_pairs", ax= ax)
ax.set_ylim(0,35)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot(data = detection_rates_df, x="time_frame", y="false_positive_rate", ax= ax)
sns.lineplot(data = detection_rates_df, x="time_frame", y="false_negative_rate", ax= ax)
ax.set_ylim(0.0,1.1)